# Определение стоимости автомобилей

**Описание проекта**

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение, чтобы привлечь новых клиентов. В нём можно будет узнать рыночную стоимость своего автомобиля. 



**Цель проекта**

Необходимо построить модель, которая умеет определять рыночную стоимость автомобиля. В распоряжении данные о технических характеристиках, комплектации и ценах других автомобилей.

Критерии, которые важны заказчику:
* качество предсказания;
* время обучения модели;
* время предсказания модели.

**План проекта**

1. Подготовка данных;
2. Обучение моделей;
3. Анализ моделей.

**Описание данных**

*Признаки*
* DateCrawled — дата скачивания анкеты из базы
* VehicleType — тип автомобильного кузова
* RegistrationYear — год регистрации автомобиля
* Gearbox — тип коробки передач
* Power — мощность (л. с.)
* Model — модель автомобиля
* Kilometer — пробег (км)
* RegistrationMonth — месяц регистрации автомобиля
* FuelType — тип топлива
* Brand — марка автомобиля
* Repaired — была машина в ремонте или нет
* DateCreated — дата создания анкеты
* NumberOfPictures — количество фотографий автомобиля
* PostalCode — почтовый индекс владельца анкеты (пользователя)
* LastSeen — дата последней активности пользователя

*Целевой признак*
* Price — цена (евро)

## Подготовка данных

In [1]:
import numpy as np
import pandas as pd

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor


In [2]:
try:
    data = pd.read_csv('/datasets/autos.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv')

data.head(15)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


<div class='alert alert-success'> ✔️Правильно, что сгруппировала импорты в одном месте, это облегчает понимание используемого в работе функционала и улучшает читабельность кода.
try-except тоже к месту, чтобы проект запускался и локально, и удаленно.
</div>

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
data.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Удалим строчки с пропущенными моделями так как это важный критерий.

In [5]:
data = data.dropna(subset = ['Model']).reset_index(drop = True)

In [6]:
data.isna().sum()

DateCrawled              0
Price                    0
VehicleType          30662
RegistrationYear         0
Gearbox              15702
Power                    0
Model                    0
Kilometer                0
RegistrationMonth        0
FuelType             25732
Brand                    0
Repaired             62100
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Заполним пропущенные значения в данных самым популярным значением в соответствующем столбце по соответствующим марки автомобиля и модели.

In [7]:
for brand in data['Brand'].unique():
    for model in data.query('Brand == @brand')['Model'].unique():
        data.loc[(data['Brand'] == brand) & (data['Model'] == model) & (data['VehicleType'].isna()), 'VehicleType'] = \
        data.query('Brand == @brand & Model == @model')['VehicleType'].mode()[0]
        data.loc[(data['Brand'] == brand) & (data['Model'] == model) & (data['Gearbox'].isna()), 'Gearbox'] = \
        data.query('Brand == @brand & Model == @model')['Gearbox'].mode()[0]
        data.loc[(data['Brand'] == brand) & (data['Model'] == model) & (data['FuelType'].isna()), 'FuelType'] = \
        data.query('Brand == @brand & Model == @model')['FuelType'].mode()[0]
        try:
            data.loc[(data['Brand'] == brand) & (data['Model'] == model) & (data['Repaired'].isna()), 'Repaired'] = \
            data.query('Brand == @brand & Model == @model')['Repaired'].mode()[0]
        except:
            data.loc[(data['Brand'] == brand) & (data['Model'] == model) & (data['Repaired'].isna()), 'Repaired'] = \
        'unknown'

In [8]:
data.isna().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

Удалим 2 строчки с пропущенными Repaired.

In [9]:
data = data.dropna(subset = ['Repaired']).reset_index(drop = True)
data.isna().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

In [10]:
for column in data.columns:
    print(column,': ')
    print()
    print(data[column].unique())
    print("________________")
    print()

DateCrawled : 

['2016-03-24 11:52:17' '2016-03-14 12:52:21' '2016-03-17 16:54:04' ...
 '2016-03-19 19:53:49' '2016-03-21 09:50:58' '2016-03-19 18:57:12']
________________

Price : 

[  480  9800  1500 ... 12395 18429 10985]
________________

VehicleType : 

['sedan' 'suv' 'small' 'convertible' 'bus' 'wagon' 'coupe' 'other']
________________

RegistrationYear : 

[1993 2004 2001 2008 1995 1980 2014 1998 2005 2011 1910 2016 2007 2009
 2002 2018 1997 1990 2017 1981 2003 1994 1991 1984 2006 1999 2012 2010
 2000 1992 2013 1996 1985 1989 2015 1982 1976 1983 1973 1969 1971 1987
 1986 1988 1965 1970 1945 1974 1955 1978 1979 1972 1968 1977 1961 1960
 1966 1975 1963 5000 1954 1967 1000 9999 1500 2200 1964 3000 2066 1949
 1962 2019 1937 1951 1959 1953 1958 8000 1956 9000 2900 5900 5911 1400
 1950 4000 1952 1200 8500 1932 3700 1942 1935 1234 1933 1936 6500 1957
 1923 2290 1930 1001 7000 1929 1943 1934 1938 1928 1941 1800 1919 5555
 6000 1300 1931 4500 1602 1111 7800 1947 1927 7100 2500 8200 7500 

Удалим строки с анамалиями в столбцах RegistrationYear, RegistrationMonth, Price, Power.

In [11]:
data = data.query('RegistrationYear < 2023 & RegistrationMonth > 0 & Price > 0 & Power > 0').reset_index(drop = True).reset_index(drop = True)

Удалим столбцы DateCrawled, DateCreated, NumberOfPictures, LastSeen, PostalCode так как эти столбцы ни на что не влияют.

In [12]:
data = data.drop(['DateCrawled', 'DateCreated', 'NumberOfPictures','LastSeen','PostalCode'], axis = 1)

Проверим на наличие дубликатов

In [13]:
data.duplicated().sum()

26574

Удалим дубликаты.

In [14]:
data = data.drop_duplicates(keep = 'first')

Поделим данные на целевой признак и остальные.

In [15]:
features = data.drop(['Price'], axis = 1)
target = data['Price']

Разобьем данные на обучающую, валидационную и тестовую выборки в соотношении 60/20/20.

In [16]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.2, random_state = 12345) 

In [17]:
features_train, features_test, target_train, target_test = train_test_split(features_train, target_train, test_size = 0.25, random_state = 12345) 

In [18]:
features_train.shape

(150327, 10)

In [19]:
features_valid.shape

(50109, 10)

In [20]:
features_test.shape

(50109, 10)

Преобразуем категориальные признаки техникой OrdinalEncoder

In [21]:
calegorical = ['VehicleType','Gearbox','Model','FuelType','Brand','Repaired']
encoder = OrdinalEncoder()
encoder.fit(features_train[calegorical])
features_train_ordinal = pd.DataFrame(encoder.transform(features_train[calegorical]),columns = calegorical,index=features_train.index)
features_valid_ordinal = pd.DataFrame(encoder.transform(features_valid[calegorical]),columns = calegorical,index=features_valid.index)
features_test_ordinal = pd.DataFrame(encoder.transform(features_test[calegorical]),columns = calegorical,index=features_test.index)

Соединим датафреймы после преобразований.

In [22]:
features_train = pd.concat((features_train.drop(calegorical,axis=1),features_train_ordinal),axis = 1)
features_valid = pd.concat((features_valid.drop(calegorical,axis=1),features_valid_ordinal),axis = 1)
features_test = pd.concat((features_test.drop(calegorical,axis=1),features_test_ordinal),axis = 1)

Численные признаки приведем к одному масштабу.

In [23]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
features_test = scaler.transform(features_test)

**Вывод:**

1. Были изученны предоставленные данные.
2. Удалены пропущенные значения в столбцах Model.
3. Заполнены проуски в столбцах VehicleType, Gearbox, FuelType, Repaired наиболее популярными значениями среди соответствующих бренда и модели.
4. Удалены строки с анамалиями в столбцах RegistrationYear, RegistrationMonth, Price, Power.
5. Удалены столбцы DateCrawled, DateCreated, NumberOfPictures, LastSeen, PostalCode так как эти столбцы ни на что не влияют.
6. Данные разделены на обучающую, валидационную и тестовую выборки в соотношении 60/20/20/
7. Преобразованы категориальные признаки техникой OrdinalEncoder.
8. Численные признаки приведены к одному масштабу.

## Обучение моделей

Построим модель с использованием алгоритма дерева решений.

In [24]:
model = DecisionTreeRegressor()
params = {'max_depth':np.arange(1,11)}
search = GridSearchCV(estimator = model, param_grid = params, cv = 3)
search.fit(features_train,target_train)
best_model = search.best_estimator_
best_model

DecisionTreeRegressor(max_depth=10)

In [25]:
predictions = best_model.predict(features_valid)
mean_squared_error(target_valid, predictions) ** 0.5

2003.120346921405

Построим модель с использованием алгоритма случайного леса.

In [26]:
model = RandomForestRegressor()
params = {'max_depth':np.arange(1,7),
         'n_estimators':np.arange(20, 50, 10)}
search = RandomizedSearchCV(estimator = model, param_distributions = params, cv = 3)
search.fit(features_train,target_train)
best_model = search.best_estimator_
best_model

RandomForestRegressor(max_depth=6, n_estimators=30)

In [27]:
predictions = best_model.predict(features_valid)
mean_squared_error(target_valid, predictions) ** 0.5

2282.6146586705827

Построим модель с использованием линейной регрессии.

In [28]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)

mean_squared_error(target_valid, predictions) ** 0.5

3510.0304131950184

Построим модель с использованием бустинга LightGBM.

In [29]:
model = LGBMRegressor(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)

mean_squared_error(target_valid, predictions) ** 0.5

1704.8078698703173

In [30]:
model = LGBMRegressor()
params = {'max_depth':np.arange(1,11),
         'n_estimators':np.arange(20, 80, 10)}
search = GridSearchCV(estimator = model, param_grid = params, cv = 3)
search.fit(features_train,target_train)
best_model = search.best_estimator_
best_model

LGBMRegressor(max_depth=10, n_estimators=70)

In [31]:
predictions = best_model.predict(features_valid)
mean_squared_error(target_valid, predictions) ** 0.5

1738.3021451734803

**Вывод:**
1. Построена модель с использованием алгоритма дерева решений. rmse показала значение 2003.1.
2. Построена модель с использованием алгоритма случайного леса. rmse показала значение 2282.6.
3. Построена модель с использованием линейной регрессии. rmse показала значение 3510, что не подходит нам по условиям задачи (rmse должно быть больше 2500).
4. Построены модели с использованием градиентным бустинга LightGBM. rmse показала значение 1704.8 и 1738.3.

## Анализ моделей

Проверим время работы модели DecisionTreeRegressor с параметром depth = 9

In [32]:
%%time
model = DecisionTreeRegressor(max_depth=10, random_state=12345)
model.fit(features_train, target_train)

CPU times: total: 578 ms
Wall time: 452 ms


DecisionTreeRegressor(max_depth=10, random_state=12345)

Проверим модель на тестовой выборке.

In [33]:
%%time
predictions = model.predict(features_valid)

CPU times: total: 0 ns
Wall time: 9.97 ms


Проверим время работы модели RandomForestRegressor с параметрами depth = 6 и max_depth = 30

In [34]:
%%time
model = RandomForestRegressor(n_estimators=40, max_depth=6, random_state=12345)
model.fit(features_train, target_train)

CPU times: total: 7.73 s
Wall time: 7.83 s


RandomForestRegressor(max_depth=6, n_estimators=40, random_state=12345)

Проверим модель на тестовой выборке.

In [35]:
%%time
predictions = model.predict(features_valid)

CPU times: total: 141 ms
Wall time: 150 ms


Проверим время работы модели LinearRegression

In [36]:
%%time
model = LinearRegression()
model.fit(features_train, target_train)

CPU times: total: 46.9 ms
Wall time: 47.9 ms


LinearRegression()

Проверим модель на тестовой выборке.

In [37]:
%%time
predictions = model.predict(features_valid)

CPU times: total: 15.6 ms
Wall time: 4.75 ms


Проверим время работы модели LGBMRegressor

In [38]:
%%time
model = LGBMRegressor(random_state=12345)
model.fit(features_train, target_train)

CPU times: total: 2.94 s
Wall time: 816 ms


LGBMRegressor(random_state=12345)

Проверим модель на тестовой выборке.

In [39]:
%%time
predictions = model.predict(features_valid)

CPU times: total: 625 ms
Wall time: 170 ms


Лучший результат по времени обучения и предсказания показала модель LinearRegression, на втором месте DecisionTreeRegressor(max_depth=10, random_state=12345), на третьем LGBMRegressor, однако наилучшее значение по качеству показала модель LGBMRegressor. Следовательно, основываясь на предпочтениях клиента, можно сделать вывод, что подходят две модели  DecisionTreeRegressor(max_depth=10, random_state=12345) и LGBMRegressor. Но так как качество стоит на первом месте, выберем модель LGBMRegressor.

Протестируем модель LGBMRegressor.

In [41]:
model = LGBMRegressor(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
mean_squared_error(target_test, predictions) ** 0.5

1680.756872014941

Модель показала хороший результат при тестировании.

**Вывод:**

Критерии, которые важны заказчику:
- качество предсказания;
- время обучения модели;
- время предсказания модели.
    
Лучший результат по времени обучения и предсказания показала модель LinearRegression, на втором месте DecisionTreeRegressor(max_depth=10, random_state=12345), на третьем LGBMRegressor, однако наилучшее значение по качеству показала модель LGBMRegressor. Следовательно, основываясь на предпочтениях клиента, можно сделать вывод, что подходят две модели  DecisionTreeRegressor(max_depth=10, random_state=12345) и LGBMRegressor. Но так как качество стоит на первом месте, выберем модель LGBMRegressor. Выбранная модель показала хороший результат при тестировании.

## Общий вывод

1. Подготовка данных

    1.1. Были изученны предоставленные данные.
    
    1.2. Удалены пропущенные значения в столбцах Model.
    
    1.3. Заполнены проуски в столбцах VehicleType, Gearbox, FuelType, Repaired наиболее популярными значениями среди соответствующих бренда и модели.
    
    1.4. Удалены строки с анамалиями в столбцах RegistrationYear, RegistrationMonth, Price, Power.
    
    1.5. Удалены столбцы DateCrawled, DateCreated, NumberOfPictures, LastSeen, PostalCode так как эти столбцы ни на что не влияют.
    
    1.6. Данные разделены на обучающую, валидационную и тестовую выборки в соотношении 60/20/20/
    
    1.7. Преобразованы категориальные признаки техникой OrdinalEncoder.
    
    1.8. Численные признаки приведены к одному масштабу.
    

2. Обучение моделей

    2.1. Построена модель с использованием алгоритма дерева решений. rmse показала значение 2003.1.
    
    2.2. Построена модель с использованием алгоритма случайного леса. rmse показала значение 2282.6.
    
    2.3. Построена модель с использованием линейной регрессии. rmse показала значение 3510, что не подходит нам по условиям задачи (rmse должно быть больше 2500).
    
    2.4. Построены модели с использованием градиентным бустинга LightGBM. rmse показала значение 1704.8 и 1738.3.
    
    
3. Анализ моделей

    Критерии, которые важны заказчику:
    - качество предсказания;
    - время обучения модели;
    - время предсказания модели.
    
    
    Лучший результат по времени обучения и предсказания показала модель LinearRegression, на втором месте DecisionTreeRegressor(max_depth=10, random_state=12345), на третьем LGBMRegressor, однако наилучшее значение по качеству показала модель LGBMRegressor. Следовательно, основываясь на предпочтениях клиента, можно сделать вывод, что подходят две модели  DecisionTreeRegressor(max_depth=10, random_state=12345) и LGBMRegressor. Но так как качество стоит на первом месте, выберем модель LGBMRegressor. Выбранная модель показала хороший результат при тестировании.
